In [1]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!unzip cat_dogs.zip

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50V2
from keras.layers import Dense, Flatten ,Dropout
from keras.models import Model, Sequential

In [3]:
target_size =(224,224)
## 이미지 파일을 불러와서 변수에 담는다.
train = ImageDataGenerator().flow_from_directory('/content/drive/MyDrive/Colab Notebooks/train', target_size =target_size, batch_size=10, seed=42)
valid = ImageDataGenerator().flow_from_directory('/content/drive/MyDrive/Colab Notebooks/valid', target_size =target_size, batch_size=30, seed=42)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.


In [4]:
## 전이학습을 위한 pretrained model load
## weights = trained data
## include_top = fc의 가중치 이용 여부
## pool = 'avg or max' - flatten과 동일한 효과 차원을 1차원으로 축소 
base_model = ResNet50V2(weights='imagenet', include_top=True, input_shape=(224,224,3), pooling='max')

102869336/102869336 [==============================] - 1s 0us/step


In [5]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

In [6]:
## pre-trained 모델과 내가 만든 fc 연결
n_model = Sequential()
n_model.add(base_model)
n_model.add(Dense(64,activation='relu'))
n_model.add(Dropout(0.5))
n_model.add(Dense(2,activation='softmax', name='softmax'))
n_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 1000)              25613800  
                                                                 
 dense (Dense)               (None, 64)                64064     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 softmax (Dense)             (None, 2)                 130       
                                                                 
Total params: 25,677,994
Trainable params: 25,632,554
Non-trainable params: 45,440
_________________________________________________________________


In [8]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint(
    'resnet50.hdf5',
    monitor = "val_loss",
    verbose = 1,
    save_best_only = True,
 )
earlystp = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
## 옵티마이저의 Adam을 불러와 learning rate 를 수기로 조정
n_model.compile(optimizer = Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
hist = n_model.fit(train, epochs=20, callbacks=[checkpoint, earlystp], validation_data=valid, verbose=1)

Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.5495
Epoch 1: val_loss improved from inf to 0.69199, saving model to resnet50.hdf5
21/21 [==============================] - 45s 2s/step - loss: 0.6922 - accuracy: 0.5495 - val_loss: 0.6920 - val_accuracy: 0.4951
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 0.6790 - accuracy: 0.7376
Epoch 2: val_loss improved from 0.69199 to 0.68298, saving model to resnet50.hdf5
21/21 [==============================] - 4s 191ms/step - loss: 0.6790 - accuracy: 0.7376 - val_loss: 0.6830 - val_accuracy: 0.6311
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 0.6654 - accuracy: 0.9010
Epoch 3: val_loss improved from 0.68298 to 0.66659, saving model to resnet50.hdf5
21/21 [==============================] - 4s 192ms/step - loss: 0.6654 - accuracy: 0.9010 - val_loss: 0.6666 - val_accuracy: 0.8058
Epoch 4/30
21/21 [==============================] - ETA: 0s - loss: 0.6570 - accuracy

In [10]:
## 전이 학습이 완료된 모델에 실제 평가를 위한 test dataset load
test = ImageDataGenerator().flow_from_directory('/content/drive/MyDrive/Colab Notebooks/test', target_size =target_size, batch_size=30)
## 평가 [loss : 0.34008756279945374, acc : 0.9467849135398865]
n_model.evaluate(test)

Found 451 images belonging to 2 classes.
16/16 [==============================] - 31s 2s/step - loss: 0.4500 - accuracy: 0.8537


[0.4500204622745514, 0.8536585569381714]

In [12]:
from keras.utils import load_img , img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np



In [27]:
## 파일 이름을 리스트에 넣어서 n개 예측
img=[1,2]
for i in img:
  img =load_img((f'/content/drive/MyDrive/Colab Notebooks/examples/{i}.jpeg'), target_size=(224,224))
  image=img_to_array(img)
  image= image.reshape(1, image.shape[0],image.shape[1],image.shape[2])
  img = preprocess_input(image)
  pred= n_model.predict(img)
  percent = pred*100
  if np.argmax(pred) ==1 :
    print(f'{percent[0][1]:.2f} %의 확률로 개')
  else : 
    print(f'{percent[0][0]:.2f} %의 확률로 고양이')

1/1 [==============================] - 0s 20ms/step
66.26 %의 확률로 개
1/1 [==============================] - 0s 20ms/step
57.02 %의 확률로 고양이


In [14]:
img=load_img(('/content/drive/MyDrive/Colab Notebooks/examples/1.jpeg'), target_size=(224,224))
image=img_to_array(img)
image= image.reshape(1, image.shape[0],image.shape[1],image.shape[2])


In [15]:
img = preprocess_input(image)

In [19]:
pred= n_model.predict(img)

1/1 [==============================] - 0s 22ms/step


In [20]:
print(pred)

[[0.33740047 0.6625995 ]]


In [17]:
## 예측된 값 확인_ 최대 확률이 반환
if np.argmax(pred) ==1 :
  print('개')
else : 
  print('고양이')

개


In [25]:
percent =pred*100


In [26]:
percent

array([[33.740047, 66.25995 ]], dtype=float32)